OBS: Na base de dados para treino enviada pelo Github, verifique se a coluna 'Datetime' foi alterada para 'Date'. Caso não tenha, o modelo não funcionara

In [1]:
# Célula 1: Instalações
!pip install streamlit pyngrok gymnasium gym_anytrading stable-baselines3 matplotlib numpy pandas --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6

In [2]:
# Célula 2: Criar backtesting_engine.py
%%writefile backtesting_engine.py

import gymnasium as gym
import gym_anytrading
from stable_baselines3 import A2C
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from gym_anytrading.envs import StocksEnv
from gym_anytrading.envs import Actions
import traceback
import os
import json

# --- Definições de Ambiente Personalizado ---
def add_signals(env_instance):
    start = env_instance.frame_bound[0] - env_instance.window_size
    end = env_instance.frame_bound[1]
    if start < 0: start = 0
    prices_df = env_instance.df.iloc[start:end]
    required_signal_cols = ['Low', 'Volume', 'SMA', 'RSI', 'OBV', 'ticker_id_norm']
    missing_signal_cols = [col for col in required_signal_cols if col not in prices_df.columns]
    if missing_signal_cols:
        raise KeyError(f"Colunas faltando em `prices_df` dentro de `add_signals`: {missing_signal_cols}. Colunas disponíveis: {prices_df.columns.tolist()}")
    prices = prices_df['Low'].to_numpy()
    signal_features = prices_df[required_signal_cols].to_numpy()
    return prices, signal_features

class MyCustomEnv(StocksEnv):
    _process_data = add_signals
    def __init__(self, df, window_size, frame_bound):
        self.num_signal_features = len(['Low', 'Volume', 'SMA', 'RSI', 'OBV', 'ticker_id_norm'])
        super().__init__(df=df, window_size=window_size, frame_bound=frame_bound)

# --- Função para formatar moeda em R$ ---
def format_brl(value, incluir_sinal_positivo=False):
    if not isinstance(value, (int, float)): return "N/A"
    prefixo = ""
    if value > 0 and incluir_sinal_positivo:
        prefixo = "+"
    valor_abs_formatado = f"{abs(value):_.2f}".replace('.', '#TEMP#').replace(',', '.').replace('#TEMP#', ',')
    if value < 0:
        return f"-R$ {valor_abs_formatado}"
    return f"{prefixo}R$ {valor_abs_formatado}"

# --- Função para Carregar Dados CSV ---
def carregar_dados_csv_master(caminho_arquivo_csv_ou_buffer):
    try:
        df = pd.read_csv(caminho_arquivo_csv_ou_buffer)
        required_cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"O CSV deve conter as colunas: {', '.join(required_cols)}")
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    except FileNotFoundError: raise FileNotFoundError(f"Arquivo CSV não encontrado: {caminho_arquivo_csv_ou_buffer}")
    except Exception as e: raise Exception(f"Erro ao carregar CSV: {str(e)}")

# --- Função de Pré-processamento e Engenharia de Features ---
def preprocess_and_feature_engineer_for_ticker(df_ticker_data_original, ticker_simbolo_log=""):
    df = df_ticker_data_original.copy()
    if df.empty: raise ValueError(f"Nenhum dado para pré-processar: {ticker_simbolo_log}.")
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    elif not isinstance(df.index, pd.DatetimeIndex): raise ValueError(f"Índice de data inválido para {ticker_simbolo_log}.")
    df.sort_index(inplace=True)
    required_ohlcv = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in required_ohlcv: df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '', regex=False), errors='coerce')
    df[required_ohlcv] = df[required_ohlcv].fillna(method='ffill').fillna(method='bfill')
    if df[required_ohlcv].isnull().values.any(): raise ValueError(f"NaNs em OHLCV para {ticker_simbolo_log}.")
    if not df.index.is_unique: df = df[~df.index.duplicated(keep='first')]

    df['SMA'] = df['Close'].rolling(window=12, min_periods=1).mean()
    delta = df['Close'].diff(); gain = delta.clip(lower=0); loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/14, adjust=False, min_periods=1).mean()
    avg_loss = loss.ewm(alpha=1/14, adjust=False, min_periods=1).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan); df['RSI'] = 100.0 - (100.0 / (1.0 + rs))
    price_diff = df['Close'].diff(); signed_volume = pd.Series(0.0, index=df.index)
    signed_volume.loc[price_diff > 0] = df['Volume']; signed_volume.loc[price_diff < 0] = -df['Volume']
    df['OBV'] = signed_volume.cumsum()
    df.fillna(0, inplace=True)
    return df

# --- Função de TESTE com Modelo Global Pré-Treinado ---
def testar_ticker_com_modelo_global(ticker_simbolo, df_dados_teste_original,
                                    modelo_global_path,
                                    ticker_to_id_map,
                                    valor_investimento_inicial_reais=10000.0):
    status_messages = []
    resultados = {
        "ticker": ticker_simbolo, "figura_grafico_avaliacao": None,
        "recompensa_total_avaliacao": "N/A", "lucro_total_percentual_avaliacao": 0.0,
        "lucro_total_reais_avaliacao": 0.0, "prejuizo_total_reais_avaliacao": 0.0,
        "saldo_final_reais_avaliacao": valor_investimento_inicial_reais,
        "valor_investimento_inicial": valor_investimento_inicial_reais,
        "tabela_qualitativa": pd.DataFrame(), "info_simulacao_treinada": "N/A",
        "erros": None, "status_messages": status_messages, "modelo_carregado": False,
        "num_buy_decisions": 0, "num_sell_decisions": 0,
        "num_good_buy_decisions": 0, "num_good_sell_decisions": 0,
        "buy_decision_hit_rate": "N/A", "sell_decision_hit_rate": "N/A"
    }
    def log_status(msg): print(msg); status_messages.append(msg)

    fig_avaliacao_obj = None
    try:
        log_status(f"Iniciando TESTE para: {ticker_simbolo} com modelo global.")
        if not os.path.exists(modelo_global_path):
            raise FileNotFoundError(f"Modelo global {modelo_global_path} não encontrado.")

        model = A2C.load(modelo_global_path)
        resultados["modelo_carregado"] = True
        log_status(f"Modelo global carregado de {modelo_global_path}.")

        df_ticker_teste_raw = df_dados_teste_original[df_dados_teste_original['Ticker'] == ticker_simbolo].copy()
        if df_ticker_teste_raw.empty: raise ValueError(f"Nenhum dado de teste para {ticker_simbolo}.")

        df_processado_teste = preprocess_and_feature_engineer_for_ticker(df_ticker_teste_raw, ticker_simbolo)

        if ticker_simbolo not in ticker_to_id_map:
            log_status(f"AVISO: Ticker {ticker_simbolo} não encontrado no ticker_to_id_map. Usando ID normalizado default 0.5.")
            ticker_id_norm = 0.5
        else:
            ticker_id_norm = ticker_to_id_map[ticker_simbolo]

        df_processado_teste['ticker_id_norm'] = ticker_id_norm
        log_status(f"Dados de TESTE para {ticker_simbolo} processados com ticker_id_norm: {ticker_id_norm:.4f}")

        total_rows = len(df_processado_teste)
        window_size_eval = 12

        start_bound_eval = window_size_eval
        end_bound_eval = total_rows - 1

        fig_avaliacao_obj, ax_avaliacao = plt.subplots(figsize=(18,7))
        if total_rows < window_size_eval + 5 or start_bound_eval >= end_bound_eval - window_size_eval +1:
            log_status(f"Dados de teste insuficientes para avaliação de {ticker_simbolo}.")
            if fig_avaliacao_obj: plt.close(fig_avaliacao_obj); fig_avaliacao_obj = None
            resultados["saldo_final_reais_avaliacao"] = valor_investimento_inicial_reais
        else:
            log_status(f"Configurando ambiente de avaliação de TESTE: frame_bound=({start_bound_eval}, {end_bound_eval})")
            env_avaliacao = MyCustomEnv(df=df_processado_teste, window_size=window_size_eval,
                                        frame_bound=(start_bound_eval, end_bound_eval))
            obs_aval, _ = env_avaliacao.reset(); log_status("Executando simulação com modelo GLOBAL...")

            num_buy_decisions_count = 0
            num_sell_decisions_count = 0
            num_good_buy_decisions_count = 0
            num_good_sell_decisions_count = 0

            while True:
                action_pred, _ = model.predict(obs_aval, deterministic=True)
                action_to_take = action_pred.item() if isinstance(action_pred, np.ndarray) and action_pred.ndim == 0 else action_pred[0] if isinstance(action_pred, np.ndarray) else action_pred

                if action_to_take == Actions.Buy.value:
                    num_buy_decisions_count += 1
                elif action_to_take == Actions.Sell.value:
                    num_sell_decisions_count += 1

                obs_aval, reward_step, terminated, truncated, info_aval = env_avaliacao.step(action_to_take)

                if reward_step > 0:
                    if action_to_take == Actions.Buy.value:
                        num_good_buy_decisions_count += 1
                    elif action_to_take == Actions.Sell.value:
                        num_good_sell_decisions_count += 1

                if terminated or truncated:
                    resultados["recompensa_total_avaliacao"] = info_aval.get('total_reward', 0.0)
                    lucro_perc = info_aval.get('total_profit', 0.0)
                    resultados["lucro_total_percentual_avaliacao"] = lucro_perc
                    lucro_liquido_reais = lucro_perc * valor_investimento_inicial_reais if isinstance(lucro_perc, (int, float)) else 0.0
                    resultados["lucro_total_reais_avaliacao"] = lucro_liquido_reais if lucro_liquido_reais >=0 else 0.0
                    resultados["prejuizo_total_reais_avaliacao"] = abs(lucro_liquido_reais) if lucro_liquido_reais < 0 else 0.0
                    resultados["saldo_final_reais_avaliacao"] = valor_investimento_inicial_reais + lucro_liquido_reais
                    resultados["info_simulacao_treinada"] = info_aval; log_status(f"Info simulação (TESTE): {info_aval}"); break

            resultados["num_buy_decisions"] = num_buy_decisions_count
            resultados["num_sell_decisions"] = num_sell_decisions_count
            resultados["num_good_buy_decisions"] = num_good_buy_decisions_count
            resultados["num_good_sell_decisions"] = num_good_sell_decisions_count

            if num_buy_decisions_count > 0:
                resultados["buy_decision_hit_rate"] = f"{(num_good_buy_decisions_count / num_buy_decisions_count) * 100:.2f}%"
            if num_sell_decisions_count > 0:
                resultados["sell_decision_hit_rate"] = f"{(num_good_sell_decisions_count / num_sell_decisions_count) * 100:.2f}%"

            log_status(f"Sinais de Compra: {resultados['num_buy_decisions']} (Boas: {resultados['num_good_buy_decisions']}, Taxa Acerto: {resultados['buy_decision_hit_rate']})")
            log_status(f"Sinais de Venda: {resultados['num_sell_decisions']} (Boas: {resultados['num_good_sell_decisions']}, Taxa Acerto: {resultados['sell_decision_hit_rate']})")

            if hasattr(env_avaliacao.unwrapped, 'render_all'):
                env_avaliacao.unwrapped.render_all();
                profit_str_title = f"{resultados['lucro_total_percentual_avaliacao']*100:.2f}%" if isinstance(resultados['lucro_total_percentual_avaliacao'], (int,float)) else "N/A %"
                reward_str_title = f"{resultados['recompensa_total_avaliacao']:.2f}" if isinstance(resultados['recompensa_total_avaliacao'], (int,float)) else "N/A"
                lucro_reais_formatado_titulo = format_brl(resultados.get('lucro_total_reais_avaliacao',0) - resultados.get('prejuizo_total_reais_avaliacao',0))
                ax_avaliacao.set_title(
                    f"Simulação Modelo GLOBAL (Ticker: {ticker_simbolo})\n"
                    f"Resultado: {profit_str_title} ({lucro_reais_formatado_titulo}) | Recompensa Agente: {reward_str_title}",
                    fontsize=10
                )
                resultados["figura_grafico_avaliacao"] = fig_avaliacao_obj
            else: log_status("render_all não disponível."); plt.close(fig_avaliacao_obj); fig_avaliacao_obj=None

        status_modelo_str = "Global Carregado e Avaliado" if resultados["modelo_carregado"] else "Erro ao carregar/Avaliação não realizada"
        tabela_dados = {
            'Métrica': ['Ticker', 'Investimento Inicial', 'Recompensa Avaliação',
                        'Resultado Avaliação (%)', 'Lucro Avaliação (R$)',
                        'Prejuízo Avaliação (R$)', 'Saldo Final Avaliação (R$)',
                        'Nº Decisões de Compra', 'Acerto Compra (Proxy %)',
                        'Nº Decisões de Venda', 'Acerto Venda (Proxy %)',
                        'Status Modelo'],
            'Valor': [
                ticker_simbolo, format_brl(valor_investimento_inicial_reais),
                f"{resultados['recompensa_total_avaliacao']:.2f}" if isinstance(resultados['recompensa_total_avaliacao'],(int,float)) else "N/A",
                f"{resultados['lucro_total_percentual_avaliacao']*100:.2f}%" if isinstance(resultados['lucro_total_percentual_avaliacao'],(int,float)) else "N/A",
                format_brl(resultados['lucro_total_reais_avaliacao']) if resultados['lucro_total_reais_avaliacao'] > 0 else format_brl(0.0),
                format_brl(resultados['prejuizo_total_reais_avaliacao']) if resultados['prejuizo_total_reais_avaliacao'] > 0 else format_brl(0.0),
                format_brl(resultados['saldo_final_reais_avaliacao']),
                resultados['num_buy_decisions'], resultados['buy_decision_hit_rate'],
                resultados['num_sell_decisions'], resultados['sell_decision_hit_rate'],
                status_modelo_str
            ]
        }
        resultados["tabela_qualitativa"] = pd.DataFrame(tabela_dados)
        log_status(f"TESTE para {ticker_simbolo} concluído.")

    except Exception as e:
        error_msg = f"Erro no TESTE de {ticker_simbolo}: {str(e)}"
        log_status(error_msg); log_status(traceback.format_exc())
        resultados["erros"] = error_msg
        if fig_avaliacao_obj and isinstance(fig_avaliacao_obj, plt.Figure): plt.close(fig_avaliacao_obj)
        resultados["figura_grafico_avaliacao"] = None
        for key_default in ["lucro_total_percentual_avaliacao", "lucro_total_reais_avaliacao", "prejuizo_total_reais_avaliacao"]:
            if resultados.get(key_default) == "N/A": resultados[key_default] = 0.0
        if resultados.get("saldo_final_reais_avaliacao") == "N/A": resultados["saldo_final_reais_avaliacao"] = valor_investimento_inicial_reais

    return resultados

if __name__ == '__main__':
    print("Este é o backtesting_engine.py.")

Writing backtesting_engine.py


In [3]:
# Célula 3: Criar train_global_model.py
%%writefile train_global_model.py

import os
import pandas as pd
import numpy as np
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
import traceback
import json
import time

try:
    from backtesting_engine import MyCustomEnv, carregar_dados_csv_master, preprocess_and_feature_engineer_for_ticker
except ImportError:
    print("ERRO IMPORT: 'backtesting_engine.py' não encontrado. Execute a célula anterior que cria este arquivo.")
    exit()

# --- Função para criar dados globais de treinamento ---
def criar_dados_globais_treinamento(df_all_data_original):
    tickers = df_all_data_original['Ticker'].unique()
    num_total_tickers = len(tickers)
    if num_total_tickers == 0:
        raise ValueError("Nenhum ticker encontrado nos dados para treinamento.")

    ticker_to_id_map_norm = {ticker: i / (num_total_tickers -1 if num_total_tickers > 1 else 1)
                             for i, ticker in enumerate(tickers)}

    map_dfs_processados_para_treino = {}
    print(f"Processando dados para {num_total_tickers} tickers para o modelo global...")

    for ticker in tickers:
        df_ticker_raw = df_all_data_original[df_all_data_original['Ticker'] == ticker]
        try:
            df_ticker_processed = preprocess_and_feature_engineer_for_ticker(df_ticker_raw, ticker)
            df_ticker_processed['ticker_id_norm'] = ticker_to_id_map_norm[ticker]
            map_dfs_processados_para_treino[ticker] = df_ticker_processed
            print(f"Dados para {ticker} processados e ticker_id_norm ({ticker_to_id_map_norm[ticker]:.4f}) adicionado.")
        except Exception as e_process:
            print(f"Erro ao processar dados para {ticker}: {e_process}. Pulando este ticker.")
            continue

    if not map_dfs_processados_para_treino:
        raise ValueError("Nenhum ticker pôde ser processado para criar o DataFrame global de treinamento.")

    print(f"{len(map_dfs_processados_para_treino)} tickers processados com sucesso para o treinamento.")
    return map_dfs_processados_para_treino, ticker_to_id_map_norm


if __name__ == "__main__":
    ARQUIVO_CSV_TREINAMENTO = "dados_desafio_v5.csv"
    MODELO_GLOBAL_SALVO_PATH = "A2C_global_model.zip"
    TICKER_ID_MAP_PATH = "ticker_to_id_map.json"
    TIMESTEPS_TOTAIS_TREINO_GLOBAL = 100000

    print(f"Iniciando script de treinamento global usando dados de: {ARQUIVO_CSV_TREINAMENTO}")
    print(f"Modelo global será salvo em: ./{MODELO_GLOBAL_SALVO_PATH}")
    print(f"Mapa de Ticker para ID será salvo em: ./{TICKER_ID_MAP_PATH}")
    print(f"Timesteps totais para o modelo global: {TIMESTEPS_TOTAIS_TREINO_GLOBAL}")

    model_global = None

    try:
        df_principal_treino = carregar_dados_csv_master(ARQUIVO_CSV_TREINAMENTO)
        map_dfs_processados_treino, ticker_to_id_map = criar_dados_globais_treinamento(df_principal_treino)

        if not map_dfs_processados_treino:
            print("ERRO: Nenhum dado de ticker foi processado com sucesso. Encerrando.")
            exit()

        with open(TICKER_ID_MAP_PATH, 'w') as f:
            json.dump(ticker_to_id_map, f, indent=4)
        print(f"Mapa Ticker->ID salvo em {TICKER_ID_MAP_PATH}")

        print("\nIniciando treinamento iterativo do modelo global...")

        tickers_para_treino_efetivo = list(map_dfs_processados_treino.keys())
        if not tickers_para_treino_efetivo:
             print("Nenhum ticker com dados processados para treinamento. Encerrando.")
             exit()

        window_size_train_global = 12

        primeiro_ticker_df = map_dfs_processados_treino[tickers_para_treino_efetivo[0]]
        if len(primeiro_ticker_df) < window_size_train_global + 20:
            print(f"Dados insuficientes para o primeiro ticker {tickers_para_treino_efetivo[0]} para inicializar o ambiente. Encerrando.")
            exit()

        env_inicial = MyCustomEnv(df=primeiro_ticker_df, window_size=window_size_train_global,
                                  frame_bound=(window_size_train_global, len(primeiro_ticker_df)-1))
        vec_env_inicial = DummyVecEnv([lambda: env_inicial])
        model_global = A2C('MlpPolicy', vec_env_inicial, verbose=1, tensorboard_log=f"./rl_tensorboard_logs_global/")

        num_tickers_validos = len(tickers_para_treino_efetivo)
        timesteps_por_ticker_iteracao = TIMESTEPS_TOTAIS_TREINO_GLOBAL // num_tickers_validos if num_tickers_validos > 0 else TIMESTEPS_TOTAIS_TREINO_GLOBAL
        if timesteps_por_ticker_iteracao < 1000: timesteps_por_ticker_iteracao = 1000
        print(f"Treinando com {timesteps_por_ticker_iteracao} timesteps por ticker (total aprox. {timesteps_por_ticker_iteracao * num_tickers_validos}).")

        for ticker_nome in tickers_para_treino_efetivo:
            print(f"\n--- Treinando modelo global com dados de {ticker_nome} por {timesteps_por_ticker_iteracao} timesteps ---")
            df_ticker_atual_proc = map_dfs_processados_treino[ticker_nome]

            if len(df_ticker_atual_proc) < window_size_train_global + 20 :
                print(f"Dados insuficientes para {ticker_nome} nesta iteração de treino. Pulando.")
                continue

            env_ticker_atual = MyCustomEnv(df=df_ticker_atual_proc, window_size=window_size_train_global,
                                           frame_bound=(window_size_train_global, len(df_ticker_atual_proc)-1) )
            vec_env_ticker_atual = DummyVecEnv([lambda: env_ticker_atual])

            model_global.set_env(vec_env_ticker_atual)
            model_global.learn(total_timesteps=timesteps_por_ticker_iteracao, reset_num_timesteps=False)

        model_global.save(MODELO_GLOBAL_SALVO_PATH)
        print(f"\n--- Modelo Global treinado e salvo em {MODELO_GLOBAL_SALVO_PATH} ---")

    except FileNotFoundError:
        print(f"ERRO CRÍTICO: Arquivo de treinamento '{ARQUIVO_CSV_TREINAMENTO}' não encontrado.")
    except Exception as e:
        print(f"Um erro crítico ocorreu no script de treinamento global: {e}")
        traceback.print_exc()

Writing train_global_model.py


In [4]:
# Célula 5: Executar o treinamento offline do modelo global
!python train_global_model.py

A saída de streaming foi truncada nas últimas 5000 linhas.
-------------------------------------

--- Treinando modelo global com dados de KLAC por 1000 timesteps ---
Logging to ./rl_tensorboard_logs_global/A2C_0
------------------------------------
| time/                 |          |
|    fps                | 478      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 587500   |
| train/                |          |
|    entropy_loss       | -0.00979 |
|    explained_variance | -0.0692  |
|    learning_rate      | 0.0007   |
|    n_updates          | 117499   |
|    policy_loss        | 0.0531   |
|    value_loss         | 0.229    |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 497       |
|    iterations         | 200       |
|    time_elapsed       | 2         |
|    total_timesteps    | 588000    |
| train/                |           |
|  

In [5]:
# Célula 6: Criar app_streamlit_global_tester.py
%%writefile app_streamlit_global_tester.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

# --- Função para formatar moeda em R$ ---
def formatar_reais(valor, incluir_sinal_positivo=False):
    if not isinstance(valor, (int, float)): return "N/A"
    prefixo = ""
    if valor > 0 and incluir_sinal_positivo: prefixo = "+"
    valor_abs_formatado = f"{abs(valor):_.2f}".replace('.', '#TEMP#').replace(',', '.').replace('#TEMP#', ',')
    if valor < 0: return f"-R$ {valor_abs_formatado}"
    return f"{prefixo}R$ {valor_abs_formatado}"

# --- Importação do Engine ---
try:
    from backtesting_engine import testar_ticker_com_modelo_global, carregar_dados_csv_master
    ENGINE_DISPONIVEL = True
except ImportError as e:
    st.error(f"Falha ao importar 'backtesting_engine.py': {e}.")
    ENGINE_DISPONIVEL = False
    def testar_ticker_com_modelo_global(ticker, df_teste, modelo_path, ticker_map, valor_investimento_inicial_reais):
        st.info(f"MODO MOCK: Simulando TESTE GLOBAL para {ticker}...")
        fig_mock, ax_mock = plt.subplots(); ax_mock.text(0.5, 0.5, f"Gráfico Mock TESTE GLOBAL {ticker}", ha='center')
        mock_lucro_perc = np.random.uniform(-0.1, 0.2); mock_lucro_liq_reais = mock_lucro_perc * valor_investimento_inicial_reais
        return {"ticker": ticker, "erros": "Engine mock.", "figura_grafico_avaliacao": fig_mock,
                "tabela_qualitativa": pd.DataFrame({
                    'Métrica':['Ticker', 'Nº Compras', 'Acerto Compra', 'Nº Vendas', 'Acerto Venda'],
                    'Valor':[ticker, 5, "60.00%", 4, "50.00%"]
                }),
                "lucro_total_percentual_avaliacao": mock_lucro_perc, "recompensa_total_avaliacao": np.random.uniform(-10,100),
                "lucro_total_reais_avaliacao": mock_lucro_liq_reais if mock_lucro_liq_reais >=0 else 0.0,
                "prejuizo_total_reais_avaliacao": abs(mock_lucro_liq_reais) if mock_lucro_liq_reais < 0 else 0.0,
                "saldo_final_reais_avaliacao": valor_investimento_inicial_reais + mock_lucro_liq_reais,
                "valor_investimento_inicial": valor_investimento_inicial_reais,
                "status_messages": [f"Engine mock para teste global de {ticker}."], "modelo_carregado": False,
                "num_buy_decisions": 5, "num_sell_decisions": 4,
                "buy_decision_hit_rate": "60.00%", "sell_decision_hit_rate": "50.00%"}
    def carregar_dados_csv_master(p):
        st.warning("Usando dados mock."); data = {'Date': pd.to_datetime(['2024-01-01']), 'Ticker': ['MOCKTEST'], 'Open': [1],'High': [1],'Low': [1],'Close': [1],'Volume': [1]}; return pd.DataFrame(data)

st.set_page_config(layout="wide", page_title="RL Global Model Tester")
st.title("🧪 Testador de Modelo Global de Trading RL (A2C)")
st.markdown("Carregue dados de TESTE, selecione um ticker e veja a simulação usando o modelo global pré-treinado.")
st.markdown("---")

if 'resultados_por_ticker_teste_global' not in st.session_state: st.session_state.resultados_por_ticker_teste_global = {}
if 'df_teste_carregado_global' not in st.session_state: st.session_state.df_teste_carregado_global = None
if 'lista_tickers_teste_global' not in st.session_state: st.session_state.lista_tickers_teste_global = []
if 'ultimo_hash_arquivo_teste_global' not in st.session_state: st.session_state.ultimo_hash_arquivo_teste_global = None
if 'ticker_ativo_para_exibicao_teste_global' not in st.session_state: st.session_state.ticker_ativo_para_exibicao_teste_global = None
if 'ticker_id_map_carregado' not in st.session_state: st.session_state.ticker_id_map_carregado = None

MODELO_GLOBAL_PATH_DEFAULT = "A2C_global_model.zip"
TICKER_ID_MAP_PATH_DEFAULT = "ticker_to_id_map.json"

if ENGINE_DISPONIVEL and st.session_state.ticker_id_map_carregado is None:
    if os.path.exists(TICKER_ID_MAP_PATH_DEFAULT):
        try:
            with open(TICKER_ID_MAP_PATH_DEFAULT, 'r') as f: st.session_state.ticker_id_map_carregado = json.load(f)
            st.sidebar.success("Mapa Ticker->ID carregado.")
        except Exception as e_map: st.sidebar.error(f"Erro ao carregar mapa Ticker->ID: {e_map}")
    else: st.sidebar.warning(f"'{TICKER_ID_MAP_PATH_DEFAULT}' não encontrado. Execute 'train_global_model.py'.")

st.sidebar.header("⚙️ Configurações de Teste (Modelo Global)")
arquivo_teste_carregado = st.sidebar.file_uploader("1. Carregue CSV com dados de TESTE", type="csv", key="file_uploader_test_global")

if arquivo_teste_carregado is not None:
    current_file_hash = arquivo_teste_carregado.name + str(arquivo_teste_carregado.size)
    if st.session_state.df_teste_carregado_global is None or current_file_hash != st.session_state.ultimo_hash_arquivo_teste_global:
        if ENGINE_DISPONIVEL:
            try:
                with st.spinner("Carregando dados de TESTE..."):
                    st.session_state.df_teste_carregado_global = carregar_dados_csv_master(arquivo_teste_carregado)
                    st.session_state.lista_tickers_teste_global = sorted(st.session_state.df_teste_carregado_global['Ticker'].unique())
                    st.session_state.resultados_por_ticker_teste_global = {}; st.session_state.ultimo_hash_arquivo_teste_global = current_file_hash
                    st.session_state.ticker_ativo_para_exibicao_teste_global = None
                st.sidebar.success(f"CSV de TESTE carregado! {len(st.session_state.lista_tickers_teste_global)} tickers.")
            except Exception as e: st.sidebar.error(f"Erro CSV TESTE: {str(e)}"); st.session_state.df_teste_carregado_global = None; st.session_state.lista_tickers_teste_global = []
        else: st.sidebar.error("Engine não disponível.")

if not ENGINE_DISPONIVEL and st.session_state.df_teste_carregado_global is None:
    st.session_state.df_teste_carregado_global = carregar_dados_csv_master("mock")
    st.session_state.lista_tickers_teste_global = sorted(st.session_state.df_teste_carregado_global['Ticker'].unique())

if st.session_state.df_teste_carregado_global is not None and st.session_state.lista_tickers_teste_global:
    st.sidebar.markdown("### Teste Individual com Modelo Global")
    ticker_para_testar_sidebar = st.sidebar.selectbox("Escolha um Ticker para TESTAR:", st.session_state.lista_tickers_teste_global, key="ticker_test_sb_global")
    valor_investimento_teste_sidebar = st.sidebar.number_input("Valor de Investimento Inicial (R$) para teste:", min_value=100.0, value=10000.0, step=100.0, format="%.2f", key="invest_test_input_global")
    caminho_modelo_global_input = st.sidebar.text_input("Caminho para o modelo GLOBAL salvo:", value=MODELO_GLOBAL_PATH_DEFAULT, key="model_global_path_input")

    if st.sidebar.button(f"🧪 Testar Modelo para {ticker_para_testar_sidebar}", use_container_width=True, key="run_test_global_button"):
        if ticker_para_testar_sidebar and ENGINE_DISPONIVEL:
            if st.session_state.ticker_id_map_carregado is None: st.sidebar.error(f"Mapa Ticker->ID não carregado.")
            elif not os.path.exists(caminho_modelo_global_input): st.sidebar.error(f"Modelo global '{caminho_modelo_global_input}' não encontrado!")
            else:
                with st.spinner(f"Testando modelo GLOBAL para {ticker_para_testar_sidebar}..."):
                    resultados_teste = testar_ticker_com_modelo_global(
                        ticker_para_testar_sidebar, st.session_state.df_teste_carregado_global,
                        modelo_global_path=caminho_modelo_global_input,
                        ticker_to_id_map=st.session_state.ticker_id_map_carregado,
                        valor_investimento_inicial_reais=valor_investimento_teste_sidebar)
                    st.session_state.resultados_por_ticker_teste_global[ticker_para_testar_sidebar] = resultados_teste
                    st.session_state.ticker_ativo_para_exibicao_teste_global = ticker_para_testar_sidebar
                st.success(f"Teste GLOBAL para {ticker_para_testar_sidebar} concluído!"); st.rerun()
        elif not ENGINE_DISPONIVEL: st.sidebar.error("Motor de backtesting não carregado.")
        else: st.sidebar.warning("Selecione um ticker para testar.")

    st.sidebar.markdown("---"); st.sidebar.markdown("### Teste em Lote (Modelo Global)")
    reprocessar_existentes_teste_lote_global = st.sidebar.checkbox("Reprocessar tickers já testados (global)?", value=False, key="reprocess_test_lote_cb_global")
    if st.sidebar.button("🌐 Testar TODOS os Tickers (Modelo Global)", use_container_width=True, key="run_all_tickers_test_global_button"):
        if st.session_state.df_teste_carregado_global is not None and st.session_state.lista_tickers_teste_global:
            if st.session_state.ticker_id_map_carregado is None: st.sidebar.error(f"Mapa Ticker->ID não carregado.")
            elif not os.path.exists(caminho_modelo_global_input): st.sidebar.error(f"Modelo global '{caminho_modelo_global_input}' não encontrado.")
            else:
                num_total_tickers_teste = len(st.session_state.lista_tickers_teste_global)
                st.sidebar.info(f"Iniciando teste em lote (global) para {num_total_tickers_teste} tickers...")
                progress_area_lote = st.container(); progress_text_area_lote = progress_area_lote.empty(); progress_bar_lote = progress_area_lote.progress(0)
                for i, ticker_nome_teste_lote in enumerate(st.session_state.lista_tickers_teste_global):
                    progress_text_area_lote.info(f"Testando {ticker_nome_teste_lote} ({i+1}/{num_total_tickers_teste})...")
                    invest_lote_teste = valor_investimento_teste_sidebar
                    if ticker_nome_teste_lote not in st.session_state.resultados_por_ticker_teste_global or reprocessar_existentes_teste_lote_global:
                        resultados_ticker_teste_lote = testar_ticker_com_modelo_global(
                            ticker_nome_teste_lote, st.session_state.df_teste_carregado_global,
                            modelo_global_path=caminho_modelo_global_input,
                            ticker_to_id_map=st.session_state.ticker_id_map_carregado,
                            valor_investimento_inicial_reais=invest_lote_teste)
                        st.session_state.resultados_por_ticker_teste_global[ticker_nome_teste_lote] = resultados_ticker_teste_lote
                    else: st.sidebar.info(f"Ticker {ticker_nome_teste_lote} já testado. Pulando.")
                    progress_bar_lote.progress((i + 1) / num_total_tickers_teste, text=f"Testado: {ticker_nome_teste_lote} ({i+1}/{num_total_tickers_teste})")
                progress_text_area_lote.success(f"Teste em lote (global) para {num_total_tickers_teste} tickers concluído!")
                progress_bar_lote.empty(); st.balloons()
                st.session_state.ticker_ativo_para_exibicao_teste_global = st.session_state.lista_tickers_teste_global[-1] if st.session_state.lista_tickers_teste_global else None
                st.rerun()
        else: st.sidebar.warning("Carregue um CSV de TESTE.")

elif not ENGINE_DISPONIVEL and st.session_state.df_teste_carregado_global is None: st.sidebar.error("Motor de backtesting não carregado.")
elif not arquivo_teste_carregado and ENGINE_DISPONIVEL: st.sidebar.info("Carregue um CSV com dados de TESTE.")

st.header("📊 Resultados do Teste com Modelo GLOBAL Pré-Treinado")
if not st.session_state.resultados_por_ticker_teste_global:
    st.info("Nenhum ticker foi testado ainda. Use o menu à esquerda.")
else:
    tickers_testados = list(st.session_state.resultados_por_ticker_teste_global.keys())
    ticker_a_exibir_teste_padrao = st.session_state.ticker_ativo_para_exibicao_teste_global
    if not ticker_a_exibir_teste_padrao or ticker_a_exibir_teste_padrao not in tickers_testados:
        if tickers_testados: ticker_a_exibir_teste_padrao = tickers_testados[0]
        else: ticker_a_exibir_teste_padrao = None
    default_idx_teste = 0
    if ticker_a_exibir_teste_padrao and tickers_testados:
        try: default_idx_teste = tickers_testados.index(ticker_a_exibir_teste_padrao)
        except ValueError: default_idx_teste = 0

    ticker_a_exibir_main_teste = st.selectbox("Resultados do Teste para (Modelo Global):", tickers_testados,
                                         index=default_idx_teste if tickers_testados else 0,
                                         key="ticker_display_test_global_sb_main")
    if ticker_a_exibir_main_teste:
        res_teste = st.session_state.resultados_por_ticker_teste_global[ticker_a_exibir_main_teste]
        st.subheader(f"Detalhes do Teste para: {res_teste.get('ticker', 'N/A')}")

        if res_teste.get("erros"): st.error(f"Erro: {res_teste['erros']}")
        elif not res_teste.get("modelo_carregado") and ENGINE_DISPONIVEL:
            st.warning("Modelo global não foi carregado, ou simulação não concluída.")

        invest_inicial_str = formatar_reais(res_teste.get('valor_investimento_inicial',0.0))
        lucro_perc_val = res_teste.get('lucro_total_percentual_avaliacao', 0.0)
        lucro_reais_val = res_teste.get('lucro_total_reais_avaliacao', 0.0)
        prejuizo_reais_val = res_teste.get('prejuizo_total_reais_avaliacao', 0.0)
        saldo_final_val = res_teste.get('saldo_final_reais_avaliacao', 0.0)
        cor_resultado = "green" if lucro_perc_val > 0 else "red" if lucro_perc_val < 0 else "gray"

        st.markdown(f"**Investimento Inicial Simulado:** {invest_inicial_str}")
        col1, col2, col3, col4 = st.columns(4)
        with col1: st.markdown(f"**Resultado (%):** <font color='{cor_resultado}'>{lucro_perc_val*100:.2f}%</font>", unsafe_allow_html=True)
        with col2: st.markdown(f"**Lucro (R$):** <font color='green'>{formatar_reais(lucro_reais_val, True)}</font>", unsafe_allow_html=True)
        with col3: st.markdown(f"**Prejuízo (R$):** <font color='red'>{formatar_reais(prejuizo_reais_val)}</font>", unsafe_allow_html=True)
        with col4:
            cor_sf = "gray"
            inv_i = res_teste.get('valor_investimento_inicial', 0.0)
            if isinstance(saldo_final_val, (int,float)) and isinstance(inv_i, (int,float)):
                if saldo_final_val > inv_i: cor_sf = "green"
                elif saldo_final_val < inv_i: cor_sf = "red"
            st.markdown(f"**Saldo Final (R$):** <font color='{cor_sf}'>{formatar_reais(saldo_final_val)}</font>", unsafe_allow_html=True)

        st.markdown("#### Gráfico: Simulação com Modelo Treinado")
        fig_teste = res_teste.get("figura_grafico_avaliacao")
        if fig_teste and isinstance(fig_teste, plt.Figure) :
            st.pyplot(fig_teste)
            st.caption("No gráfico: Pontos verdes indicam COMPRA, Pontos vermelhos indicam VENDA.")
        else: st.caption("Gráfico de simulação de teste não disponível.")

        st.markdown("#### Tabela Qualitativa do Teste")
        tabela_q_teste = res_teste.get("tabela_qualitativa")
        if isinstance(tabela_q_teste, pd.DataFrame) and not tabela_q_teste.empty:
            st.table(tabela_q_teste)
        else: st.caption("Tabela qualitativa não disponível.")

        with st.expander("Logs de Processamento do Teste", expanded=False):
            status_msgs_teste = res_teste.get("status_messages", [])
            if status_msgs_teste:
                for msg in status_msgs_teste: st.text(msg)
            else: st.caption("Nenhum log.")

    if len(st.session_state.resultados_por_ticker_teste_global) > 0:
        st.markdown("---"); st.header("Resumo Geral dos Testes (Modelo Global)")
        dados_resumo = []; lucros_perc = []; lucros_reais_liq = []; num_l, num_p = 0,0
        total_inv = 0; total_sf = 0
        for t, r_d in st.session_state.resultados_por_ticker_teste_global.items():
            inv_i_val = r_d.get('valor_investimento_inicial', 0.0)
            l_p_val = r_d.get('lucro_total_percentual_avaliacao', 0.0)
            l_r_liq = (r_d.get('lucro_total_reais_avaliacao', 0.0) - r_d.get('prejuizo_total_reais_avaliacao', 0.0))
            sf_val = r_d.get('saldo_final_reais_avaliacao', inv_i_val)
            total_inv += inv_i_val; lucros_reais_liq.append(l_r_liq); total_sf += sf_val
            l_p_s = "N/A"
            if isinstance(l_p_val, (int,float)):
                l_p_s = f"{l_p_val*100:.2f}%"; lucros_perc.append(l_p_val)
                if l_p_val > 0: num_l+=1
                elif l_p_val < 0: num_p+=1
            dados_resumo.append({"Ticker": t, "Invest. Inicial (R$)": formatar_reais(inv_i_val),
                                 "Resultado (%)": l_p_s,
                                 "Resultado Líquido (R$)": formatar_reais(l_r_liq, True),
                                 "Saldo Final (R$)": formatar_reais(sf_val),
                                 "Erro": "Sim" if r_d.get("erros") else "Não"})
        df_res_geral = pd.DataFrame(dados_resumo)
        st.markdown("#### Performance por Ticker (Teste com Modelo Global):"); st.dataframe(df_res_geral)
        st.markdown(f"**Total Investido (testes):** {formatar_reais(total_inv)}")
        if lucros_perc:
            lucro_med_p = np.mean(lucros_perc) * 100
            cor_lmp = "green" if lucro_med_p > 0 else "red" if lucro_med_p < 0 else "gray"
            rotulo_lmp = "Lucro Médio Geral (%)" if lucro_med_p >= 0 else "Prejuízo Médio Geral (%)"
            st.markdown(f"**{rotulo_lmp}:** <font color='{cor_lmp}'>{abs(lucro_med_p):.2f}%</font>", unsafe_allow_html=True)
        if lucros_reais_liq:
            res_liq_total = np.sum(lucros_reais_liq)
            cor_rlt = "green" if res_liq_total > 0 else "red" if res_liq_total < 0 else "gray"
            rotulo_rlt = "Resultado Líquido Total Agregado (R$)" if res_liq_total >=0 else "Prejuízo Líquido Total Agregado (R$)"
            st.markdown(f"**{rotulo_rlt}:** <font color='{cor_rlt}'>{formatar_reais(abs(res_liq_total))}</font>", unsafe_allow_html=True)
        st.markdown(f"**Saldo Final Total Agregado (R$):** {formatar_reais(total_sf)}")
        st.markdown(f"Tickers com Lucro: <font color='green'>{num_l}</font> | Tickers com Perda: <font color='red'>{num_p}</font>", unsafe_allow_html=True)

Writing app_streamlit_global_tester.py


In [8]:
# Célula 7: Executar o app_streamlit_global_tester.py com ngrok
!pip install pyngrok -q
from pyngrok import ngrok, conf
import os
import time
import traceback

NGROK_PLACEHOLDER = "SEU_AUTHTOKEN_AQUI"
NGROK_AUTH_TOKEN = "2xByl1WHSIJX6JEJNxGmcLfdEqh_685aSd3PVsR8hs1sYANcm"
if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != NGROK_PLACEHOLDER:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("Authtoken do ngrok configurado.")
else:
    print("AVISO: Authtoken do ngrok não fornecido ou é o placeholder.")

os.system("kill $(ps aux | grep 'streamlit run app_streamlit_global_tester.py' | grep -v grep | awk '{print $2}') > /dev/null 2>&1")
os.system("kill $(ps aux | grep 'ngrok http 8501' | grep -v grep | awk '{print $2}') > /dev/null 2>&1")
os.system("pkill -f ngrok > /dev/null 2>&1")
time.sleep(2)

os.system("nohup streamlit run app_streamlit_global_tester.py --server.port 8501 --server.headless true &")

print("Aplicativo Streamlit GLOBAL TESTER (app_streamlit_global_tester.py) iniciado...")
time.sleep(15)

try:
    for tunnel in ngrok.get_tunnels():
        try: ngrok.disconnect(tunnel.public_url)
        except Exception: pass

    public_url = ngrok.connect(8501, proto="http")
    print("------------------------------------------------------------------------------------")
    print(f" ✅ Seu aplicativo TESTADOR GLOBAL Streamlit está rodando! Acesse aqui: {public_url} ")
    print("------------------------------------------------------------------------------------")
except Exception as e:
    print(f" Erro ao conectar com ngrok: {e}"); print(traceback.format_exc())
    print("\n🔴🔴🔴 FALHA AO INICIAR O TÚNEL NGROK. Verifique o erro ERR_NGROK_108 e seu painel ngrok.com/agents 🔴🔴🔴")

Authtoken do ngrok configurado.
Aplicativo Streamlit GLOBAL TESTER (app_streamlit_global_tester.py) iniciado...
------------------------------------------------------------------------------------
 ✅ Seu aplicativo TESTADOR GLOBAL Streamlit está rodando! Acesse aqui: NgrokTunnel: "https://2623-34-106-168-227.ngrok-free.app" -> "http://localhost:8501" 
------------------------------------------------------------------------------------
